In [2]:
# setup
import warnings;
warnings.filterwarnings('ignore'); #tensorflow gives me weird stuff
import numpy as np;
import tensorflow as tf;
from numpy import matmul as mul
from numpy.linalg import norm as norm
tf.enable_eager_execution()

In [3]:
def hadamard(matrices, skip_matrix = -1):
    #matrices is a list of numpy ndarrays (shape of all matrices must be the same)
    shp = matrices[0].shape
    #Commented out for performance increase
#     for m in matrices: 
#         if m.shape != shp:
#             return None
    ret = np.ones(shp)
    for i in range(shp[0]):
        for j in range(shp[1]):
            for num,k in enumerate(matrices):
                if(num != skip_matrix):
                    ret[i][j] *= k[i][j]
                    if ret[i][j] == 0: #just a lil optimization
                        break;
    return ret

In [4]:
#MTTKRP helper methods

def calc_indices(t, dims, n):
    #t is the target row we want (zero indexed)
    #dims is the dimensions of the tensor
    #n is the factor matrix we dont want to include
    
    #first get cumulative products on our factors
    prods = [1]
    nd = len(dims)
    for i in range(nd-1,-1,-1):
        if i!=n:
            p = prods[-1] * dims[i]
            prods.append(p)
    prods.reverse()
    prods = prods[1:]
    ret = [] #the coefficients of the index we want
    for i in range(nd - 1):
        n = t // prods[i];
        ret.append(n)
        t -= (prods[i] * n)
    return ret

def khatri_rao_at_ij(factors, i, j, dims, n):
    #dims is the list of dimensions, factors is the actual factor matrix, 
    #factors is the list of all factor matrices
    #gives you the khatri rao product at a certain index
    product = 1
    i_vals = calc_indices(i, dims, n)
    for num, factor in enumerate(factors):
        if num == n:
            pass
        if num < n:
            product *= factors[num][i_vals[num]][j]
        if num > n:
            product *= factors[num][i_vals[num-1]][j]
    return product

def find_index_unfolded(dims, n, idx):
    #finds the coordinates of the unfolded tensor element given the coordinates of the not-unfolded tensor
    j = 0;
    nd = len(dims)
    prod = 1;
    
    for i in range(nd):
        if i != n:
            j += idx[i] * prod
            prod *= dims[i]
    
    return (idx[n], j)

In [5]:
def MTTKRP(X, factors, weights, n,rank,dims):
    # Matricized Tensor Times Khatri Rao Product
    # X unfolded is short and fat, khatri-rao is tall and skinny
    # output is short and skinny
    
    nd = len(dims)
    output = np.zeros((dims[n],rank))
    indices = X.indices.numpy()
    values = X.values.numpy()
    
    for l in range(len(values)):
        cur_index = indices[l]
        cur_value = values[l]
        i,j = find_index_unfolded(dims, n, cur_index) 
        # gets the index of this element if we did a mode n unfolding of X
        
        for r in range(rank):
            output[i][r] += cur_value * khatri_rao_at_ij(factors,j,r,dims,n)
    
    for r in range(rank): #multiply by weights
        output[:,r]*=weights[0][r]
          
    return output

In [10]:
def cp_als(X, rank, iterations = 3):
    
    dims = X.shape.as_list()
    nd = len(dims)
    factors = [np.ones((d,rank)) for d in dims]
    l = np.ones((1,rank))
    
    for iteration in range(iterations): 
        for n in range(nd):
            h = mul(l.T,l)
            print(h)
            for i,f in enumerate(factors):
                if i != n:
                    h *= mul(f.T,f) 
            vinv = np.linalg.pinv(h)
            mk = MTTKRP(X, factors, l, n, rank, dims)
            An = mul(mk,vinv)
            for i in range(rank):
                col = An[:,i]
                weight = norm(col)
                # print( " weight: ", weight, 'col: ', col)
                if(abs(weight)> 0.000001):
                    An[:,i]/=weight
                    l[0][i]*=weight
            factors[n] = An
            
    return l, factors

In [11]:
indices = [[1,0,0],[1,0,1],[1,0,2],[1,1,0],[1,1,1],[1,1,2],[2,0,0],[2,0,1],[2,0,2],[2,1,0],[2,1,1],[2,1,2],[3,0,0],[3,0,1],[3,0,2],[3,1,0],[3,1,1],[3,1,2]]
values = [16.0, 12.0, 10.0, 64.0, 48.0, 40.0, 24.0, 18.0, 15.0, 96.0, 72.0, 60.0, 56.0, 42.0, 35.0, 224.0, 168.0, 140.0]
shape = [4,2,3]

st = tf.SparseTensor(indices=indices, values=values, dense_shape=shape)
tf.sparse.to_dense(st)

<tf.Tensor: id=11, shape=(4, 2, 3), dtype=float32, numpy=
array([[[  0.,   0.,   0.],
        [  0.,   0.,   0.]],

       [[ 16.,  12.,  10.],
        [ 64.,  48.,  40.]],

       [[ 24.,  18.,  15.],
        [ 96.,  72.,  60.]],

       [[ 56.,  42.,  35.],
        [224., 168., 140.]]], dtype=float32)>

In [12]:
st_rank = 1
factors = cp_als(st, st_rank, 1000)
factors

[[1.]]
[[15543.05555556]]
[[16503.28315412]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[

[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.0056

[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.0056

[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.9791

[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.0056

[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.97912713]]
[[72284.00564706]]
[[46799.708]]
[[124842.9791

(array([[353.33125977]]), [array([[0.        ],
         [0.25400025],
         [0.38100038],
         [0.88900089]]), array([[0.24253563],
         [0.9701425 ]]), array([[0.71554175],
         [0.53665631],
         [0.4472136 ]])])

In [13]:
def expand(factors, weights, dim_no, cur_idx, cur_prod, all_vals, rank):
    #this method just writes to all values, so all values needs to be saved somewhere
    if dim_no == len(factors):
        value = 0;
        for r in range(rank):
            value += cur_prod[r] * weights[0][r]
        if(value != 0.0):
            all_vals.append((cur_idx,value)) 
    else:
        cur_fact = factors[dim_no]
        for i in range(len(cur_fact)): # go through all rows
            cp = np.ndarray.copy(cur_prod);
            for r in range(rank): # go through each rank
                cp[r] *= cur_fact[i][r]
            expand(factors, weights, dim_no + 1, cur_idx + [i], cp, all_vals, rank)

In [14]:
def rebuild_sp_tensor_from_factors(factors, weights, dimensions, rank):
    #print(factors)
    all_values = []
    expand(factors, weights, 0, [], np.ones(rank), all_values, rank)
    #print(all_values)
    indices = [a[0] for a in all_values]
    values = [a[1] for a in all_values]
    shape = dimensions
    st = tf.SparseTensor(indices=indices, values=values, dense_shape=shape)
    return st

In [15]:
rst = rebuild_sp_tensor_from_factors(factors[1], factors[0], shape, st_rank)

In [16]:
tf.sparse.to_dense(rst)

<tf.Tensor: id=6018, shape=(4, 2, 3), dtype=float64, numpy=
array([[[  0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ]],

       [[ 15.57495256,  11.68121442,   9.73434535],
        [ 62.29981025,  46.72485769,  38.9373814 ]],

       [[ 23.36242884,  17.52182163,  14.60151803],
        [ 93.44971537,  70.08728653,  58.40607211]],

       [[ 54.51233397,  40.88425047,  34.07020873],
        [218.04933586, 163.5370019 , 136.28083491]]])>

In [17]:
tf.sparse.to_dense(st)

<tf.Tensor: id=6021, shape=(4, 2, 3), dtype=float32, numpy=
array([[[  0.,   0.,   0.],
        [  0.,   0.,   0.]],

       [[ 16.,  12.,  10.],
        [ 64.,  48.,  40.]],

       [[ 24.,  18.,  15.],
        [ 96.,  72.,  60.]],

       [[ 56.,  42.,  35.],
        [224., 168., 140.]]], dtype=float32)>